## 전처리 체크용

- get feature names

In [1]:
#!pip install brewer2mpl
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; 
warnings.filterwarnings(action='ignore')

# preprocessing
import numpy as np
import pandas as pd

# warning 삭제용
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

In [3]:
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../src') # 상위경로 추가
from data import *

/tmp/ipykernel_3646380/3209079668.py:4: DtypeWarning: Columns (43,279) have mixed types.Specify dtype option on import or set low_memory=False.
  from data import *


In [4]:
from sklearn.model_selection import train_test_split

target = "g181a297"
#target = "survived"

def split(df, target, test_size=0.3):
    
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    print("X_train.shape:", X_train.shape)
    print("X_test.shape:", X_test.shape)
    print("y_train.shape:", y_train.shape)
    print("y_test.shape:", y_test.shape)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split(df, target)

X_train.shape: (7367, 82)
X_test.shape: (3158, 82)
y_train.shape: (7367,)
y_test.shape: (3158,)


In [124]:
# import onehotencoder

import category_encoders as ce


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# column tranformer
from sklearn.compose import ColumnTransformer

# pipeline
from sklearn.pipeline import Pipeline

In [363]:
feature_dict =  {
      'g181pid':'id', # id 
       'g181majorcat':'majorcat', # 전공계열
       'g181sex':'sex_cat', # 성별   
       'g181birthy':'birth_date', # 출생년
       'g181age':'age_num', # 연령
       'g181graduy':'graduy_date', #졸업년
### 사업체 관련       
        'g181a001':'year_start_date', # 현일자리 시작년
        'g181a002':'month_start_date', # 현일자리 시작월
        'g181a004_10':'ind_cat', # 일자리 산업 대분류
        'g181a010':'corp_worker_cat', # 기업체 종사자 수 # 결측값문제로 categorical로 변경
        'g181a011':'biz_worker_cat', # 사업체 종사자 수
        'g181a018':'tw_hour', # 출근시간_시간
        'g181a019':'tw_min', # 출근시간_분
        'g181a116':'workday_num', # 주당 정규 근로일
        'g181a117':'worktime_num', # 주당정규 근로시간
        'g181a118':'worktime_ex_num', # 주당초과 근로시간
        'g181a119': 'holy_work_num', # 월평균 휴일근로
        'g181a020': 'biztype_cat', # 사업체형태
       'g181a022':'regular_cat',     # 정규직 비정규직 여부
### 혜택관련
        'g181a390':'voluntary_cat',     # 일자리 형태 자발, 비자발 여부
        'g181a035':'shift_cat' ,    # 교대제 여부
        'g181a038':'pension_cat',     # 퇴직금 제공 여부
        'g181a039':'payed_vc_cat',     # 제공여부 2- 유급휴가
        'g181a043':'maternity_cat',     #  6- 육아휴직
        'g181a045':'overtime_pay_cat',     # 8- 시간 외 수당
        'g181a046':'bonus_cat',     # 9- 상여금
        'g181a048':'weekly_hl_cat',     # 11- 유급주휴
        'g181a392':'baby_vc_cat',     # 12- 산전후휴가
        'g181a120':'wage_type_cat',     # 급여 형태 구분
        'g181a122':'month_wage_num',     # 월 평균 근로소득
        'g181a126':'sat_wage_num',     # 만족도-임금
        'g181a127':'sat_stable_num',     # 만족도-고용안정성
        'g181a128':'sat_work_num',     # 만족도-직무내용
        'g181a129':'sat_env_num',     # 만족도-근무환경
        'g181a130':'sat_wt_num',     # 만족도-노동시간
        'g181a131':'sat_potential_num',     # 만족도-발전가능성
        'g181a132':'sat_relation_num',     # 만족도-인간관계
        'g181a133':'sat_welfare_num',     # 만족도-복리후생
        'g181a134':'sat_hr_num',     # 만족도-인사체계
        'g181a135':'sat_rep1_num',     # 만족도-사회적평판-일
        'g181a136':'sat_auto_num',     # 만족도-자율성 및 권한
        'g181a137':'sat_rep2_num',     # 만족도-일자리-사회적 평판
        'g181a138':'sat_fit_num',     # 만족도-적성흥미일치도
        'g181a139':'sat_edu_num',     # 만족도-직무관련 교육
        'g181a140':'sat_general_num',     # 만족도-일자리_전반적만족도
        'g181a141':'sat_work-general_num',     # 만족도-업무_전반적만족도
        'g181a142':'edu-fit_num',     # 교육수준-일수준일치정도
        'g181a143':'skill-fit_num',     # 일기술수준-본인기술수준일치정도
        'g181a144':'major-fit_num',     # 주전공일치정도
        'g181a146':'major_help_num',     # 전공지식업무도움정도
        'g181a158':'ins_1_num',     # 보험-국민연금
        'g181a159':'ins_2_num',     # 보험-특수직역연금
        'g181a160':'ins_3_num',     # 보험-건강보험
        'g181a161':'ins_4_num',     # 보험-고용보험
        'g181a162':'ins_5_num',     # 보험-산재보험
        'g181a189':'seeking_time_num',     # 구직활동경험기간-개월
        'g181a283':'adjust_difficulty_cat',# 적응시어려움여부     
        'g181a285':'job_offer_cat',     # 다른일자리제의여부
        'g181a297':'turnover_intention',     # 이직준비 여부: target
        'g181g001':'graduate_cat',     # 대학원 경험유무
        'g181l001':'train_cat',     # 취업훈련경험유무
        'g181q001':'health_num',     # 현재 견강상태
        'g181q004':'smoke_cat',     # 흡연여부
        'g181q006':'drink_num,',     # 음주빈도
        'g181q015':'lifesat_personal',     # 삶의만족도-개인적 측면
        'g181q016':'lifesat_relational',     # 삶의만족도-관계적 측면
        'g181q017':'lifesat_group',     # 삶의만족도-소속집단
        'g181q018':'emg_joy_num',     # 감정빈도-즐거운
        'g181q019':'emg_happy_num',     # 감정빈도-행복한
        'g181q020':'emg_comfort_num',     # 감정빈도-편안한
        'g181q021':'emb_irr_num',     # 감정빈도-짜증나는
        'g181q022':'emb_negative_num',     # 감정빈도-부정적인
        'g181q023':'emb_spiritless',     # 감정빈도-무기력한
        'g181p001':'marriage_cat',     # 혼인여부
        'g181p008':'child_cat',     # 부양자녀 유무
        'g181p036':'parent_asset_cat',     # 부모님 자산규모
        'g181p046':'livetype_cat',     # 거주형태
        'g181p041':'support_cat',      # 경제적 지원을 받고 있는지
        # subset에서 추가한 변수들
        'worker_type':'worker_type_cat',
        'regular_worker':'regular_worker_cat',
        'turnover_exp':'turnover_exp_num', # 이직 경험
        'work_exp':'work_exp_cat'      
}


import numpy as np
# make pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer


class Preprocess():
    """
    X_train, X_test
    """
    
    def __init__(self):
        """
        
        """
        print("Preprocessing Class")
        self.numeric_df = None
        self.cat_df = None
        self.ordinal_col = ['biz_worker_cat','parent_asset_cat']
        
    
    def get_dtypes(self,data):
        # onehot 할 피처
        #self.numeric_df = data.select_dtypes(include=['number'])
        #self.cat_df = data.select_dtypes(exclude=['number'])
        self.cat_col = [col for col in data.columns.to_list() if col.endswith('cat') ] 
        self.onehot_col = [col for col in self.cat_col if col not in self.ordinal_col]
        self.numeric_col = [col for col in data.columns.to_list() if col not in self.onehot_col and col not in self.ordinal_col]
        
    def col_type_df(self,data):
        df = pd.DataFrame
        
        all_cols = [col for col in data.columns.to_list()]
        df = pd.DataFrame({"col":all_cols})
        
        col_cond = [
            (df['col'] in self.onehot_col) ,
            (df['col'] in self.numeric_col),
            (df['col'] in self.ordinal_col)
        ]
        col_type = ["category","numeric","ordinal"]
        df["type"] = np.select(col_cond,col_type,default="other")
                
        return df
        
                 
    def engineer(self,data):
        
        # id 칼럼 제거
        data = data.drop(['g181pid'], axis=1)
        
        # 결측값 처리
        data.replace(to_replace=-1, value=np.nan, inplace=True)
    
        # 이직경험 빈도 추가
        turnover_cond = [
        (data['g181d001'].isnull()==True ), # 첫 직장 전 취업한적 없음(알바포함)
        (data['g181d001']==1) & (data['g181d006']==1), # 알바 경험 있음
        (data['g181d001']==1) & (data['g181d006']==2) & (data['g181e001']==2), # 전직장 있음, 1번이직
        (data['g181e001']== 1)] # 전직장 2개 이상 있음
        choices = [0, 1, 2, 3]

        data['turnover_exp'] = np.select(turnover_cond, choices,default=3)
    
        data.drop(['g181d001','g181d006','g181e001'], axis=1, inplace=True)

        # 알바이외 근로경험 여부
        data['work_exp'] = np.where(data['turnover_exp'].isin([0,1]),0,1)
    
        # 구직활동기간 결측값 0으로 처리
    
        data['g181a189'] = data['g181a189'].fillna(0)
    
        # feature name 변경 (map)
    
        data.columns = data.columns.map(feature_dict)
    
    
        data = data.drop(data[data['month_wage_num'] == -1].index) # 급여 모르는 경우 제거
    
        data['work_year'] = 2019 - data['year_start_date']  # 근무기간
    
        data['work_time_num'] = data['tw_min'] + data['tw_hour'] # 출근소요시간
    
        # 보험 수
        insurances_col = [col for col in data if col.startswith('ins')]
        data['insurances_num'] = 0
        for col in insurances_col:
            data['temp'] = np.where(data[col] == 1, 1, 0)
            data['insurances_num'] += data['temp']
    
        # 회사 전반적 만족도
        biz_sat_col = [col for col in data if col.startswith('sat')]
        data['biz_sat'] = data[biz_sat_col].sum(axis=1)
    
        # 긍정적 감정
        pos_col = [col for col in data if col.startswith('emg') ] 
        data['pos'] = data[pos_col].sum(axis=1)
    
        # 부정적 감정
        neg_col = [col for col in data if col.startswith('neg') ]
        data['neg'] = data[neg_col].sum(axis=1)
    
        # 삶의 만족도
        lifesat_col = [col for col in data if col.startswith('lifesat')]
        data['lifesat'] = data[lifesat_col].sum(axis=1)
        
        # 혜택 수
        benefit_col  =['pension_cat',     # 퇴직금 제공 여부
            'payed_vc_cat',     # 제공여부 2- 유급휴가
            'maternity_cat',     #  6- 육아휴직
            'overtime_pay_cat',     # 8- 시간 외 수당
            'bonus_cat',     # 9- 상여금
            'weekly_hl_cat',     # 11- 유급주휴
            'baby_vc_cat'] # 출산휴가

        data['benefit_num'] = 0

        for cols in benefit_col:
            data['temp'] = np.where(data[cols]==1, 1, 0)
            data['benefit_num'] += data['temp']
        
        # 결측값 0으로 처리(구직활동기간이 0이기에)    
        data['seeking_time_num'] = data['seeking_time_num'].fillna(0)
    
        data.drop(columns='temp',axis=1,inplace=True) # temp 삭제
        data.drop(data[data.month_wage_num==-1].index,inplace=True) # 급여 모르는 경우 제거
    
        return data
    
    def drop_col(self,data,remove_col):
        
        all_cols = [self.numeric_col,self.onehot_col,self.ordinal_col]
        
        for col in all_cols:
            if remove_col in col:
                col.remove(remove_col)
            else:
                continue
        
        data.drop(columns = remove_col,axis=1,inplace=True)
        
        return data
    

    
    def base_pipline(self,data):
        
      
        """
        - categorical_feature imputation 고려
        - OneHotEncoder 
            - handle_unknown = 'ignore' 옵션 -> specifically useful if you don't know all possible categories
            - sparse = False 옵션 -> 기본값은 True. 리턴값을 sparse matrix에서 array로 변환
            
                
        ohe_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-2)),
            ('one-hot', OneHotEncoder(use_cat_names=True, handle_unknown='ignore'))
        ])
        
        ord_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-2)),
            ('ordinal', OrdinalEncoder(handle_unknown='ignore'))
        ])
        
        num_pipeline = Pipeline(steps=[
            ('impute', SimpleImputer(strategy='median')),
            ('scale', StandardScaler())
        ])
        
        preprocess_pipeline = ColumnTransformer(
            transformers=[
                ('ohe', ohe_pipeline, self.onehot_col),
                ('ord', ord_pipeline, self.ordinal_col),
                ('num', num_pipeline, self.numeric_col)
                ]
        )
        
        return preprocess_pipeline.fit_transform(data)

        """
        #make_pipeline(StandardScaler(), GaussianNB(priors=None))
        
        ohe_pipeline = make_pipeline(ce.OneHotEncoder(use_cat_names=True, handle_unknown='ignore'),SimpleImputer(strategy='most_frequent'))
        ord_pipeline = make_pipeline(ce.OrdinalEncoder(handle_unknown='ignore'),SimpleImputer(strategy='most_frequent'))
        num_pipeline = make_pipeline(StandardScaler(),SimpleImputer(strategy='median'))
       
       
        ohe_pipeline = make_pipeline(ce.OneHotEncoder(use_cat_names=True, handle_unknown='ignore'))
        ord_pipeline = make_pipeline(ce.OrdinalEncoder(handle_unknown='ignore'))
        num_pipeline = make_pipeline(StandardScaler())
       
        
        preprocess_pipeline = ColumnTransformer(
            transformers=[
                ('num', num_pipeline, self.numeric_col),
                ('ohe', ohe_pipeline, self.onehot_col),
                ('ord', ord_pipeline,self.ordinal_col)
            ]
        )

        pipe_new = Pipeline(steps=[
            ('ohe',ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',cols=self.onehot_col)),
            ('ord',ce.OrdinalEncoder(handle_unknown='ignore',cols=self.ordinal_col)),
            ('scaler',StandardScaler()),
            #('impute_num',SimpleImputer(strategy='median',cols=self.numeric_col)),
            ('impute_cat',SimpleImputer(strategy='most_frequent'))
            ]
        )
        
        ct = make_column_transformer(
             (num_pipeline, self.numeric_col),
             (ohe_pipeline, self.onehot_col),
             (ord_pipeline,self.ordinal_col),
             remainder= 'passthrough'
        )
        from sklearn.preprocessing import OneHotEncoder
        from sklearn.preprocessing import OrdinalEncoder
        # decision tree 를 사용할 경우 scalling이 필요하지 않음
        
        ohe_pipeline_1 = make_pipeline(ce.OneHotEncoder(handle_unknown='ignore'))
        #ord_pipeline_1 = make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder(handle_unknown='ignore'))
        #ord_pipeline_1 = make_pipeline(ce.OrdinalEncoder(handle_unknown='ignore'))
        ct_sk = make_column_transformer(
             (ohe_pipeline_1, self.onehot_col),
             remainder= 'passthrough'
        )
        #preprocess_pipeline
    
        return ct_sk
    

In [274]:
num_train = X_train.shape[0]

X = pd.concat([X_train, X_test], axis=0)
feature_col = X.columns

preprocess = Preprocess()

X_p=preprocess.engineer(X)
preprocess.get_dtypes(X_p)
X_p_col = X_p.columns

Preprocessing Class


In [215]:
X_p

,majorcat,sex_cat,birth_date,age_num,graduy_date,year_start_date,month_start_date,ind_cat,corp_worker_cat,biz_worker_cat,...,turnover_exp_num,work_exp_cat,work_year,work_time_num,insurances_num,biz_sat,pos,neg,lifesat,benefit_num
712,1,1,1990,29.166667,2018,2015.0,10.0,15.0,9.0,5.0,...,0,0,4.0,40.0,4,64.0,14,0.0,18,6
6800,4,1,1993,26.166667,2018,2017.0,12.0,8.0,9.0,7.0,...,0,0,2.0,10.0,4,80.0,21,0.0,21,7
12915,6,1,1994,24.833333,2018,2019.0,4.0,3.0,7.0,3.0,...,2,1,0.0,30.0,3,56.0,12,0.0,21,7
11758,4,1,1992,26.833333,2018,2018.0,4.0,3.0,4.0,4.0,...,0,0,1.0,1.0,4,66.0,21,0.0,18,6
13237,1,1,1991,28.500000,2018,2018.0,1.0,15.0,NaN,4.0,...,0,0,1.0,25.0,3,61.0,14,0.0,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,2,2,1994,25.000000,2018,2019.0,6.0,7.0,1.0,1.0,...,2,1,0.0,10.0,4,76.0,18,0.0,21,4
170,4,1,1994,25.666667,2018,2018.0,1.0,6.0,5.0,1.0,...,0,0,1.0,5.0,4,67.0,14,0.0,18,2
6284,2,2,1994,25.666667,2017,2017.0,8.0,11.0,8.0,7.0,...,0,0,2.0,30.0,4,57.0,15,0.0,18,6
7140,4,1,1993,25.750000,2018,2019.0,3.0,16.0,4.0,4.0,...,1,0,0.0,1.0,3,52.0,18,0.0,15,5


In [497]:
use_onehot = preprocess.onehot_col
use_num_col = preprocess.numeric_col
use_ord = preprocess.ordinal_col

# ord 없애기

# Ordinal 은 가능한 numeric으로 바꿔서 처리
# Tlqkf 비용이 너무 많이 든다.


"""
고려가능한 방법

1. 파이프라인 사용 -> 중간에 컬럼 리스트만 뽑기

2. column transformer 사용 . use snippet .https://stackoverflow.com/questions/54646709/sklearn-pipeline-get-feature-names-after-onehotencode-in-columntransformer

3. kaggle 참고 -> feature_importance class 사용

"""


use_num_col = use_num_col+use_ord

print(len(use_onehot+use_num_col))

88


In [389]:
class DFSimpleImputer(SimpleImputer):
    """A wrapper around `SimpleImputer` to return data frames with columns.
    """

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns=self.columns)

In [390]:
ohe_pipeline = make_pipeline(DFSimpleImputer(strategy='most_frequent'),ce.OneHotEncoder(use_cat_names=True, handle_unknown='ignore'))
ord_pipeline = make_pipeline(DFSimpleImputer(strategy='most_frequent'),ce.OrdinalEncoder(handle_unknown='ignore'))
num_pipeline = make_pipeline(StandardScaler(),SimpleImputer(strategy='median'))

In [496]:
transformers = [
    ('ohe', ohe_pipeline, use_onehot),
    ('ord', ord_pipeline, use_ord),
 #   ('num', num_pipeline, use_num_col)
]


pipe4 = Pipeline(steps=[
    ('onehot',ce.OneHotEncoder(use_cat_names=True,cols=use_onehot)),
    ('ordinal',ce.OrdinalEncoder(cols=use_ord))    
    ])


pipe5 = Pipeline(steps=[
    ('impute',DFSimpleImputer(strategy='most_frequent')),
    ('onehot',ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore')),
    ])

pipe5_5 = Pipeline(steps=[
    ('impute',DFSimpleImputer(strategy='most_frequent')),
    ('ordinal',ce.OrdinalEncoder()),    
    ])


pipe5_6 = Pipeline(steps=[
    ('impute',DFSimpleImputer(strategy='most_frequent')),
    ('onehot',ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',cols=use_onehot))
    ])

pipe6 = Pipeline(steps=[
    ('scaler',StandardScaler()),
    ('impute_num',SimpleImputer(strategy='most_frequent'))])


new_transformers = [
    ('cat',pipe5,use_onehot),
    ('num',pipe6,use_num_col) ]

combined_pipe = ColumnTransformer(new_transformers,remainder='passthrough')

In [475]:
print(use_onehot)
print(use_ord)

['majorcat', 'sex_cat', 'ind_cat', 'corp_worker_cat', 'biztype_cat', 'regular_cat', 'voluntary_cat', 'shift_cat', 'pension_cat', 'payed_vc_cat', 'maternity_cat', 'overtime_pay_cat', 'bonus_cat', 'weekly_hl_cat', 'baby_vc_cat', 'wage_type_cat', 'adjust_difficulty_cat', 'job_offer_cat', 'graduate_cat', 'train_cat', 'smoke_cat', 'marriage_cat', 'child_cat', 'livetype_cat', 'support_cat', 'worker_type_cat', 'regular_worker_cat', 'work_exp_cat']
['biz_worker_cat', 'parent_asset_cat']


In [476]:
combined_pipe.fit_transform(X_p)

array([[ 1.        ,  0.        ,  0.        , ...,  0.34633228,
         0.03789796,  0.46282267],
       [ 0.        ,  1.        ,  0.        , ...,  0.86499297,
         0.86206401,  1.15301207],
       [ 0.        ,  0.        ,  1.        , ...,  0.86499297,
        -0.78626808, -0.22736673],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.34633228,
         0.86206401,  1.15301207],
       [ 0.        ,  1.        ,  0.        , ..., -0.1723284 ,
        -0.37418506,  1.84320148],
       [ 0.        ,  0.        ,  0.        , ...,  0.86499297,
         0.03789796,  1.84320148]])

In [495]:
pipe5.fit_transform(X_p)

majorcat_1.0  majorcat_4.0  majorcat_6.0  majorcat_2.0  majorcat_5.0  \
0                 1             0             0             0             0   
1                 0             1             0             0             0   
2                 0             0             1             0             0   
3                 0             1             0             0             0   
4                 1             0             0             0             0   
...             ...           ...           ...           ...           ...   
10520             0             0             0             1             0   
10521             0             1             0             0             0   
10522             0             0             0             1             0   
10523             0             1             0             0             0   
10524             0             0             0             1             0   

       majorcat_3.0  majorcat_7.0  sex_cat_1.0  sex_cat_2.0  \
0                 0             0            1            0   
1                 0             0            1            0   
2                 0             0            1            0   
3                 0             0            1            0   
4                 0             0            1            0   
...             ...           ...          ...          ...   
10520             0             0            0            1   
10521             0             0            1            0   
10522             0             0            0            1   
10523             0             0            1            0   
10524             0             0            1            0   

       birth_date_1990.0  birth_date_1993.0  birth_date_1994.0  \
0                      1                  0                  0   
1                      0                  1                  0   
2                      0                  0                  1   
3                      0                  0                  0   
4                      0                  0                  0   
...                  ...                ...                ...   
10520                  0                  0                  1   
10521                  0                  0                  1   
10522                  0                  0                  1   
10523                  0                  1                  0   
10524                  0                  1                  0   

       birth_date_1992.0  birth_date_1991.0  birth_date_1987.0  \
0                      0                  0                  0   
1                      0                  0                  0   
2                      0                  0                  0   
3                      1                  0                  0   
4                      0                  1                  0   
...                  ...                ...                ...   
10520                  0                  0                  0   
10521                  0                  0                  0   
10522                  0                  0                  0   
10523                  0                  0                  0   
10524                  0                  0                  0   

       birth_date_1995.0  birth_date_1997.0  birth_date_1972.0  \
0                      0                  0                  0   
1                      0                  0                  0   
2                      0                  0                  0   
3                      0                  0                  0   
4                      0                  0                  0   
...                  ...                ...                ...   
10520                  0                  0                  0   
10521                  0                  0                  0   
10522                  0                  0                  0   
10523                  0                  0            

In [490]:
  def get_feature_names(column_transformer, verbose=None):  

        """

        Get the column names from the a ColumnTransformer containing transformers & pipelines

        Parameters
        ----------
        verbose : a boolean indicating whether to print summaries. 
            default = False


        Returns
        -------
        a list of the correct feature names

        Note: 
        If the ColumnTransformer contains Pipelines and if one of the transformers in the Pipeline is adding completely new columns, 
        it must come last in the pipeline. For example, OneHotEncoder, MissingIndicator & SimpleImputer(add_indicator=True) add columns 
        to the dataset that didn't exist before, so there should come last in the Pipeline.

        Inspiration: https://github.com/scikit-learn/scikit-learn/issues/12525 

        """

        if verbose is None:
            pass
        if verbose: print('''\n\n---------\nRunning get_feature_names\n---------\n''')
        
        column_transformer = column_transformer        
        assert isinstance(column_transformer, ColumnTransformer), "Input isn't a ColumnTransformer"
        #check_is_fitted(column_transformer)

        new_feature_names, transformer_list = [], []

        for i, transformer_item in enumerate(column_transformer.transformers_): 
            
            transformer_name, transformer, orig_feature_names = transformer_item
            orig_feature_names = list(orig_feature_names)
            
            if verbose: 
                print('\n\n', i, '. Transformer/Pipeline: ', transformer_name, ',', 
                      transformer.__class__.__name__, '\n')
                print('\tn_orig_feature_names:', len(orig_feature_names))

            if transformer == 'drop':
                    
                continue
                
            if isinstance(transformer, Pipeline):
                # if pipeline, get the last transformer in the Pipeline
                transformer = transformer.steps[-1][1]

            if hasattr(transformer, 'get_feature_names'):

                if 'input_features' in transformer.get_feature_names.__code__.co_varnames:

                    names = list(transformer.get_feature_names(orig_feature_names))

                else:

                    names = list(transformer.get_feature_names())

            elif hasattr(transformer,'indicator_') and transformer.add_indicator:
                # is this transformer one of the imputers & did it call the MissingIndicator?

                missing_indicator_indices = transformer.indicator_.features_
                missing_indicators = [orig_feature_names[idx] + '_missing_flag'\
                                      for idx in missing_indicator_indices]
                names = orig_feature_names + missing_indicators

            elif hasattr(transformer,'features_'):
                # is this a MissingIndicator class? 
                missing_indicator_indices = transformer.features_
                missing_indicators = [orig_feature_names[idx] + '_missing_flag'\
                                      for idx in missing_indicator_indices]

            else:

                names = orig_feature_names

            if verbose: 
                print('\tn_new_features:', len(names))
                print('\tnew_features:\n', names)

            new_feature_names.extend(names)
            transformer_list.extend([transformer_name] * len(names))
        
        #self.transformer_list, self.column_transformer_features = transformer_list,\
        #                                                            new_feature_names

        return new_feature_names

In [491]:
new_feat=get_feature_names(combined_pipe)

In [492]:
len(new_feat)
print(new_feat)

['majorcat_1.0', 'majorcat_4.0', 'majorcat_6.0', 'majorcat_2.0', 'majorcat_5.0', 'majorcat_3.0', 'majorcat_7.0', 'sex_cat_1.0', 'sex_cat_2.0', 'ind_cat_15.0', 'ind_cat_8.0', 'ind_cat_3.0', 'ind_cat_7.0', 'ind_cat_13.0', 'ind_cat_17.0', 'ind_cat_12.0', 'ind_cat_11.0', 'ind_cat_10.0', 'ind_cat_16.0', 'ind_cat_4.0', 'ind_cat_19.0', 'ind_cat_18.0', 'ind_cat_9.0', 'ind_cat_6.0', 'ind_cat_14.0', 'ind_cat_1.0', 'ind_cat_21.0', 'ind_cat_5.0', 'ind_cat_2.0', 'corp_worker_cat_9.0', 'corp_worker_cat_7.0', 'corp_worker_cat_4.0', 'corp_worker_cat_5.0', 'corp_worker_cat_8.0', 'corp_worker_cat_6.0', 'corp_worker_cat_2.0', 'corp_worker_cat_3.0', 'corp_worker_cat_1.0', 'biztype_cat_5.0', 'biztype_cat_3.0', 'biztype_cat_1.0', 'biztype_cat_2.0', 'biztype_cat_6.0', 'biztype_cat_4.0', 'biztype_cat_7.0', 'biztype_cat_9.0', 'regular_cat_2.0', 'regular_cat_1.0', 'voluntary_cat_1.0', 'voluntary_cat_2.0', 'shift_cat_1.0', 'shift_cat_2.0', 'pension_cat_1.0', 'pension_cat_2.0', 'pension_cat_3.0', 'payed_vc_cat_1.

In [493]:
pipe5.transform(X_p)

NotFittedError: This DFSimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [377]:
new_X_p=pipe4.fit_transform(X_p)
new_X_p_2 = pipe5.fit_transform(X_p)

print(new_X_p.shape)
print(new_X_p.columns)
print(new_X_p_2.shape)

(10525, 171)
Index(['majorcat_1.0', 'majorcat_4.0', 'majorcat_6.0', 'majorcat_2.0',
       'majorcat_5.0', 'majorcat_3.0', 'majorcat_7.0', 'sex_cat_1.0',
       'sex_cat_2.0', 'birth_date',
       ...
       'work_exp_cat_0.0', 'work_exp_cat_1.0', 'work_year', 'work_time_num',
       'insurances_num', 'biz_sat', 'pos', 'neg', 'lifesat', 'benefit_num'],
      dtype='object', length=171)
(10525, 171)


In [379]:
temp_col = new_X_p.columns

pd.DataFrame(new_X_p_2)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1990.0,29.166667,2018.0,2015.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,40.0,4.0,48.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,300.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,12.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,0.0,4.0,6.0,6.0,6.0,5.0,5.0,4.0,5.0,4.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,40.0,4.0,64.0,14.0,0.0,18.0,6.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1993.0,26.166667,2018.0,2017.0,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,5.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,330.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,5.0,0.0,1.0,4.0,7.0,7.0,7.0,7.0,7.0,7.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,10.0,4.0,80.0,21.0,0.0,21.0,7.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1994.0,24.833333,2018.0,2019.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,30.0,5.0,48.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,199.0,4.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,5.0,1.0,3.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,4.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,4.0,6.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,30.0,3.0,56.0,12.0,0.0,21.0,7.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1992.0,26.833333,2018.0,2018.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,5.0,40.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,230.0,4.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,5.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,0.0,1.0,4.0,6.0,6.0,6.0,7.0,7.0,7.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,4.0,66.0,21.0,0.0,18.0,6.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1991.0,28.500000,2018.0,2018.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,25.0,5.0,40.0,9.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,200.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,4.0,3.0,3.0,2.0,3.0,3.0,1.0,1.0,3.0,1.0,18.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,2.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,3.0,3.0,1.

In [361]:
pd.set_option('display.max_columns', None)
new_X_p

,majorcat_1.0,majorcat_4.0,majorcat_6.0,majorcat_2.0,majorcat_5.0,majorcat_3.0,majorcat_7.0,sex_cat_1.0,sex_cat_2.0,birth_date,age_num,graduy_date,year_start_date,month_start_date,ind_cat_15.0,ind_cat_8.0,ind_cat_3.0,ind_cat_7.0,ind_cat_13.0,ind_cat_17.0,ind_cat_12.0,ind_cat_11.0,ind_cat_10.0,ind_cat_16.0,ind_cat_4.0,ind_cat_19.0,ind_cat_18.0,ind_cat_9.0,ind_cat_6.0,ind_cat_14.0,ind_cat_1.0,ind_cat_21.0,ind_cat_5.0,ind_cat_nan,ind_cat_2.0,corp_worker_cat_9.0,corp_worker_cat_7.0,corp_worker_cat_4.0,corp_worker_cat_nan,corp_worker_cat_5.0,corp_worker_cat_8.0,corp_worker_cat_6.0,corp_worker_cat_2.0,corp_worker_cat_3.0,corp_worker_cat_1.0,biz_worker_cat,tw_hour,tw_min,workday_num,worktime_num,worktime_ex_num,holy_work_num,biztype_cat_5.0,biztype_cat_3.0,biztype_cat_1.0,biztype_cat_2.0,biztype_cat_6.0,biztype_cat_4.0,biztype_cat_7.0,biztype_cat_9.0,regular_cat_2.0,regular_cat_1.0,voluntary_cat_1.0,voluntary_cat_2.0,shift_cat_1.0,shift_cat_2.0,pension_cat_1.0,pension_cat_2.0,pension_cat_3.0,payed_vc_cat_1.0,payed_vc_cat_2.0,payed_vc_cat_3.0,maternity_cat_1.0,maternity_cat_3.0,maternity_cat_2.0,overtime_pay_cat_1.0,overtime_pay_cat_2.0,overtime_pay_cat_3.0,bonus_cat_1.0,bonus_cat_2.0,bonus_cat_3.0,weekly_hl_cat_3.0,weekly_hl_cat_1.0,weekly_hl_cat_2.0,baby_vc_cat_1.0,baby_vc_cat_3.0,baby_vc_cat_2.0,wage_type_cat_2.0,wage_type_cat_1.0,wage_type_cat_5.0,wage_type_cat_4.0,wage_type_cat_3.0,wage_type_cat_nan,month_wage_num,sat_wage_num,sat_stable_num,sat_work_num,sat_env_num,sat_wt_num,sat_potential_num,sat_relation_num,sat_welfare_num,sat_hr_num,sat_rep1_num,sat_auto_num,sat_rep2_num,sat_fit_num,sat_edu_num,sat_general_num,sat_work-general_num,edu-fit_num,skill-fit_num,major-fit_num,major_help_num,ins_1_num,ins_2_num,ins_3_num,ins_4_num,ins_5_num,seeking_time_num,adjust_difficulty_cat_2.0,adjust_difficulty_cat_1.0,job_offer_cat_2.0,job_offer_cat_1.0,graduate_cat_2.0,graduate_cat_1.0,train_cat_2.0,train_cat_1.0,health_num,smoke_cat_2.0,smoke_cat_1.0,"drink_num,",lifesat_personal,lifesat_relational,lifesat_group,emg_joy_num,emg_happy_num,emg_comfort_num,emb_irr_num,emb_negative_num,emb_spiritless,marriage_cat_1.0,marriage_cat_2.0,marriage_cat_4.0,marriage_cat_3.0,child_cat_2.0,child_cat_1.0,child_cat_nan,parent_asset_cat,livetype_cat_1.0,livetype_cat_3.0,livetype_cat_5.0,livetype_cat_4.0,livetype_cat_2.0,livetype_cat_6.0,support_cat_2.0,support_cat_1.0,support_cat_nan,worker_type_cat_employed,regular_worker_cat_yes,turnover_exp_num,work_exp_cat_0.0,work_exp_cat_1.0,work_year,work_time_num,insurances_num,biz_sat,pos,neg,lifesat,benefit_num
712,1,0,0,0,0,0,0,1,0,1990,29.166667,2018,2015.0,10.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.0,40.0,4.0,48.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,300.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,12.0,1,0,1,0,1,0,1,0,4,1,0,4,6,6,6,5,5,4,5,4,4,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,1,0,4.0,40.0,4,64.0,14,0.0,18,6
6800,0,1,0,0,0,0,0,1,0,1993,26.166667,2018,2017.0,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.0,10.0,5.0,40.0,0.0,0.0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,330.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1,0,1,0,0,1,1,0,5,0,1,4,7,7,7,7,7,7,4,4,1,1,0,0,0,1,0,0,2,0,1,0,0,0,0,1,0,0,1,1,0,1,0,2.0,10.0,4,80.0,21,0.0,21,7
12915,0,0,1,0,0,0,0,1,0,1994,24.833333,2018,2019.0,4.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0.0,30.0,5.0,48.0,2.0,1.0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,199.0,4.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,5.0,1.0,3.0,1.0,1.0,3.0,0.0,1,0,1,0,1,0,1,0,4,0,1,4,7,7,7,4,4,4,4,4,6,1,0,0,0,1,0,0,3,1,0,0,0,0,0,1,0,0,1,1,2,0,1,0.0,30.0,3,56.0,12,0.0,21,7
11758,0,1,0,0,0,0,0,1,0,1992,26.833333,2018,2018.0,4.0,0,0,1,0,0,0,0,0,0,0,

In [445]:
combined_pipe.fit_transform(X_p)

array([[ 1.,  0.,  0., ...,  0., 18.,  6.],
       [ 0.,  1.,  0., ...,  0., 21.,  7.],
       [ 0.,  0.,  1., ...,  0., 21.,  7.],
       ...,
       [ 0.,  0.,  0., ...,  0., 18.,  6.],
       [ 0.,  1.,  0., ...,  0., 15.,  5.],
       [ 0.,  0.,  0., ...,  0., 21.,  7.]])

In [438]:
a =combined_pipe.transformers_[0][1][1].get_feature_names()
b = combined_pipe.transformers_[1][1][1].get_feature_names()
c = combined_pipe.transformers_[2][1]
print(len(a))
print(len(b))


106
2


AttributeError: 'ColumnTransformer' object has no attribute 'remainder_'

In [433]:
print(len(use_num_col))

58


In [354]:
combined_pipe.transformers_[1][1].get_feature_names()

AttributeError: 'Pipeline' object has no attribute 'get_feature_names'

In [350]:
combined_pipe.fit_transform(X_p)

array([[ 1.        ,  1.        , 15.        , ...,  0.        ,
         0.62187837,  0.34633228],
       [ 4.        ,  1.        ,  8.        , ...,  0.        ,
         1.57433324,  0.86499297],
       [ 6.        ,  1.        ,  3.        , ...,  0.        ,
         1.57433324,  0.86499297],
       ...,
       [ 2.        ,  2.        , 11.        , ...,  0.        ,
         0.62187837,  0.34633228],
       [ 4.        ,  1.        , 16.        , ...,  0.        ,
        -0.3305765 , -0.1723284 ],
       [ 2.        ,  1.        , 15.        , ...,  0.        ,
         1.57433324,  0.86499297]])

In [352]:
len(dfd)

88

In [342]:
from sklearn.feature_selection import SelectorMixin
from sklearn.feature_extraction.text import _VectorizerMixin


def get_feature_out(estimator, feature_in):
    if hasattr(estimator,'get_feature_names'):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f'vec_{f}' \
                for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in


def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []

    for name, estimator, features in ct.transformers_:
        if name!='remainder':
            if isinstance(estimator, Pipeline):
                current_features = features
                for step in estimator:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(estimator, features)
            output_features.extend(features_out)
        elif estimator=='passthrough':
            output_features.extend(ct._feature_names_in[features])
                
    return output_features



In [349]:
print(dfd)

['onehotencoder__majorcat', 'onehotencoder__sex_cat', 'onehotencoder__ind_cat', 'onehotencoder__corp_worker_cat', 'onehotencoder__biztype_cat', 'onehotencoder__regular_cat', 'onehotencoder__voluntary_cat', 'onehotencoder__shift_cat', 'onehotencoder__pension_cat', 'onehotencoder__payed_vc_cat', 'onehotencoder__maternity_cat', 'onehotencoder__overtime_pay_cat', 'onehotencoder__bonus_cat', 'onehotencoder__weekly_hl_cat', 'onehotencoder__baby_vc_cat', 'onehotencoder__wage_type_cat', 'onehotencoder__adjust_difficulty_cat', 'onehotencoder__job_offer_cat', 'onehotencoder__graduate_cat', 'onehotencoder__train_cat', 'onehotencoder__smoke_cat', 'onehotencoder__marriage_cat', 'onehotencoder__child_cat', 'onehotencoder__livetype_cat', 'onehotencoder__support_cat', 'onehotencoder__worker_type_cat_employed', 'onehotencoder__regular_worker_cat_yes', 'onehotencoder__work_exp_cat', 'ordinalencoder__biz_worker_cat', 'ordinalencoder__parent_asset_cat', 'num__birth_date', 'num__age_num', 'num__graduy_date

In [261]:
combined_pipe.fit(X_p)

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                use_cat_names=True))]),
                                 ['majorcat', 'sex_cat', 'ind_cat',
                                  'corp_worker_cat', 'biztype_cat',
                                  'regular_cat', 'voluntary_cat', 'shift_cat',
                                  'pension_cat', 'payed_vc_cat',...
                                  'worktime_num', 'worktime_ex_num',
                                  'holy_work_num', 'month_wage_num',
                                  'sat_wage_num', 'sat_stable_num',
                         

In [269]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer

train = pd.DataFrame({'age': [23,12, 12, np.nan],
                      'Gender': ['M','F', np.nan, 'F'],
                      'income': ['high','low','low','medium'],
                      'sales': [10000, 100020, 110000, 100],
                      'foo' : [1,0,0,1],
                      'text': ['I will test this',
                               'need to write more sentence',
                               'want to keep it simple',
                               'hope you got that these sentences are junk'],
                      'y': [0,1,1,1]})
numeric_columns = ['age']
cat_columns     = ['Gender','income']

numeric_pipeline = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
cat_pipeline     = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
text_pipeline = make_pipeline(CountVectorizer(), SelectKBest(k=5))

transformers = [
    ('num', numeric_pipeline, numeric_columns),
    ('cat', cat_pipeline, cat_columns),
    ('text', text_pipeline, 'text'),
    ('simple_transformer', MinMaxScaler(), ['sales']),
]

combined_pipe = ColumnTransformer(
    transformers, remainder='passthrough')

transformed_data = combined_pipe.fit_transform(
    train.drop('y',1), train['y'])

/tmp/ipykernel_3646380/555468090.py:40: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train.drop('y',1), train['y'])
/home/jinheonyoon/anaconda3/envs/datamind/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [270]:
pd.DataFrame(transformed_data, 
             columns=get_ct_feature_names(combined_pipe))

/home/jinheonyoon/anaconda3/envs/datamind/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: 'ColumnTransformer' object has no attribute '_feature_names_in'

In [268]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)

Linux-5.15.12-1-MANJARO-x86_64-with-glibc2.17
Python 3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0]
NumPy 1.21.5
SciPy 1.7.3
Scikit-Learn 1.0.2


In [262]:
get_ct_feature_names(combined_pipe)

TypeError: get_feature_names() takes 1 positional argument but 2 were given

In [238]:
temp_ohe = ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',cols=use_onehot)
temp_ord = ce.OrdinalEncoder(handle_unknown='ignore',cols=use_ord)

temp_pipe = Pipeline(steps=[
    ('ohe',ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',cols=use_onehot)),
    ('impute',SimpleImputer(strategy='most_frequent'))
])
    
    
    

In [239]:
temp_pipe.fit(X_p)

Pipeline(steps=[('ohe',
                 OneHotEncoder(cols=['majorcat', 'sex_cat', 'ind_cat',
                                     'corp_worker_cat', 'biztype_cat',
                                     'regular_cat', 'voluntary_cat',
                                     'shift_cat', 'pension_cat', 'payed_vc_cat',
                                     'maternity_cat', 'overtime_pay_cat',
                                     'bonus_cat', 'weekly_hl_cat',
                                     'baby_vc_cat', 'wage_type_cat',
                                     'adjust_difficulty_cat', 'job_offer_cat',
                                     'graduate_cat', 'train_cat', 'smoke_cat',
                                     'marriage_cat', 'child_cat',
                                     'livetype_cat', 'support_cat',
                                     'worker_type_cat', 'regular_worker_cat',
                                     'work_exp_cat'],
                               handle_unknown='ign

In [241]:
ss=temp_pipe.transform(X_p)

In [244]:
ss.shape

(10525, 171)

In [219]:
ct.transform(X_p).shape

(10525, 88)

In [248]:
ss_col = temp_pipe.get_feature_names()

AttributeError: 'Pipeline' object has no attribute 'get_feature_names'

In [220]:
ar=ct.get_feature_names()

/home/jinheonyoon/anaconda3/envs/datamind/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: Transformer pipeline (type Pipeline) does not provide get_feature_names.

In [ ]:
preprocess.base_pipline()

In [65]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

df = pd.DataFrame({'brand': ['aaaa', 'asdfasdf', 'sadfds', 'NaN'],
                   'category': ['asdf', 'asfa', 'asdfas', 'as'],
                   'num1': [1, 1, 0, 0],
                   'target': [0.2, 0.11, 1.34, 1.123]})

numeric_features = ['num1']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['brand', 'category']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor',  LinearRegression())])
clf.fit(df.drop('target', 1), df['target'])

clf.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['onehot'].get_feature_names(categorical_features)

/tmp/ipykernel_3646380/583511642.py:31: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  clf.fit(df.drop('target', 1), df['target'])
/home/jinheonyoon/anaconda3/envs/datamind/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['brand_NaN', 'brand_aaaa', 'brand_asdfasdf', 'brand_sadfds',
       'category_as', 'category_asdf', 'category_asdfas', 'category_asfa'],
      dtype=object)

In [12]:
ld.df.head()

,majorcat,sex_cat,birth_date,age_num,graduy_date,year_start_date,month_start_date,ind_cat,corp_worker_cat,biz_worker_cat,...,turnover_exp_cat,work_exp_cat,work_year,work_time_num,insurances_num,biz_sat,pos,neg,lifesat,benefit_num
712,1,1,1990,29.166667,2018,2015.0,10.0,15.0,9.0,5.0,...,0,0,4.0,40.0,4,64.0,14,0.0,18,6
6800,4,1,1993,26.166667,2018,2017.0,12.0,8.0,9.0,7.0,...,0,0,2.0,10.0,4,80.0,21,0.0,21,7
12915,6,1,1994,24.833333,2018,2019.0,4.0,3.0,7.0,3.0,...,2,1,0.0,30.0,3,56.0,12,0.0,21,7
11758,4,1,1992,26.833333,2018,2018.0,4.0,3.0,4.0,4.0,...,0,0,1.0,1.0,4,66.0,21,0.0,18,6
13237,1,1,1991,28.500000,2018,2018.0,1.0,15.0,NaN,4.0,...,0,0,1.0,25.0,3,61.0,14,0.0,15,7


In [14]:
diag=ld.diagnose()

In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(diag)

                         dtype  missing_value_cnt  missing_value_ratio  \
majorcat                 int64                  0             0.000000   
sex_cat                  int64                  0             0.000000   
birth_date               int64                  0             0.000000   
age_num                float64                  0             0.000000   
graduy_date              int64                  0             0.000000   
year_start_date        float64                  0             0.000000   
month_start_date       float64                  0             0.000000   
ind_cat                float64                  8             0.076010   
corp_worker_cat        float64               1409            13.387173   
biz_worker_cat         float64                641             6.090261   
tw_hour                float64                  0             0.000000   
tw_min                 float64                  0             0.000000   
workday_num            float64        